In [ ]:
# --------------------------------
# Only run if you work in Colab. 
# --------------------------------

# Importera filer direkt i notebooken föredras när man jobbar på en delad fil där flera konton kan logga in samtidigt

#from google.colab import files
#uploaded_files = files.upload()

# Om man jobbar på en egen fil som bara du loggar in på kan du mounta din drive. Då kommer notebooken åt din drive i filträdet och den 
# kan hitta filerna direkt från din drive. Använd i så fall de två bortkommenterade raderna nedanför.


    # Running locally on Windows: set your data folder here
path = r"C:\Users\fabia\AH1023\Projekt\Projekt_0\Demo 2\data\\"

In [10]:
drive.mount('/content/drive')

NameError: name 'drive' is not defined

## Installerar bibliotek som inte är förinstallerade i Colab

In [ ]:
# --------------------------------
# Only run if you work in Colab. 
# --------------------------------
# Otherwise install the packages to your environment with uv see below (or pip if you work with virtual environments directly)
!pip install geopandas
!sudo apt install libspatialindex-dev
!pip install rtree
!pip install keplergl

Om du istället kör på din egen dator så installera de nödvändiga paketen med följande kommando i terminalen i VS Code (längst ner i fönstret)

```
uv add jupyter numpy pandas geopandas folium branca matplotlib
```

`matplotlib`, `folium` och `branca` behövs för funktionen explore() vi använder nedan.

`jupyter` behövs för att notebooken ska funka i Visual Studio Code.

De andra paketen är till för att manipulera data i dataframes.

## Importerar bibliotek

In [ ]:
import geopandas as gpd
import pickle
from keplergl import KeplerGl
from IPython.display import HTML, display

# Set local Windows data folder (no leading spaces)
path = r"C:\Users\fabia\AH1023\Projekt\Projekt_0\Demo 2\data"
path = os.path.normpath(path)  # normalize path separators

# If running in Colab, mount if available (safe guard)
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
except Exception:
    pass

# Validate folder and list files
if not os.path.isdir(path):
    raise FileNotFoundError(f"Data folder not found: {path!r}. Verify the path exists.")

print("Files in data folder:")
for f in sorted(os.listdir(path)):
    print("  ", f)

# Required shapefile base names
shp1 = "deso_b1_ume"
shp2 = "deso_a2_ume"

def check_shapefile_components(folder, base):
    comps = [base + ext for ext in (".shp", ".shx", ".dbf")]
    missing = [c for c in comps if not os.path.exists(os.path.join(folder, c))]
    return missing

missing1 = check_shapefile_components(path, shp1)
missing2 = check_shapefile_components(path, shp2)

if missing1:
    raise FileNotFoundError(f"Missing components for {shp1}: {missing1}")
if missing2:
    raise FileNotFoundError(f"Missing components for {shp2}: {missing2}")

# Read files using os.path.join and handle errors
try:
    B1 = gpd.read_file(os.path.join(path, shp1 + ".shp"))
    A2 = gpd.read_file(os.path.join(path, shp2 + ".shp"))
except Exception as e:
    raise RuntimeError(f"Failed to read shapefiles: {e}")

# Reproject to WGS84 (lon/lat)
B1.to_crs(4326, inplace=True)
A2.to_crs(4326, inplace=True)

def show_df(df, df_title):
    display(HTML("<h2>" + df_title + "</h2>"), df.head(), HTML('<br><br><br>'))

show_df(B1, "Befolkning")
show_df(A2, "Arbetande befolkning")
# ...existing code...

c:\Users\fabia\miniconda3\envs\geo\Lib\site-packages\keplergl\keplergl.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


Files in data folder:
   deso_a2_ume.cpg
   deso_a2_ume.dbf
   deso_a2_ume.prj
   deso_a2_ume.qpj
   deso_a2_ume.shp
   deso_a2_ume.shx
   deso_b1_ume.cpg
   deso_b1_ume.dbf
   deso_b1_ume.prj
   deso_b1_ume.qpj
   deso_b1_ume.shp
   deso_b1_ume.shx
   kepler_config.pkl


,ogc_fid,deso,alder0_6,alder7_15,alder16_19,alder20_24,alder25_44,alder45_64,alder65_w,totalt,geometry
0,5022,2480A0080,259,263,98,90,624,592,429,2355,"POLYGON ((20.32892 64.03356, 20.31428 64.03195..."
1,5023,2480A0090,127,164,52,39,356,320,284,1342,"POLYGON ((20.64183 64.00944, 20.62268 64.00109..."
2,5030,2480C1350,128,179,80,60,339,404,344,1534,"POLYGON ((20.31906 63.83298, 20.31904 63.83293..."
3,5031,2480C1390,52,22,19,134,394,218,153,992,"POLYGON ((20.28038 63.83529, 20.28018 63.83511..."
4,5032,2480C1250,58,47,43,172,511,321,292,1444,"POLYGON ((20.25727 63.8278, 20.25704 63.82752,..."


,ogc_fid,deso,offentliga,naringsliv,totalt,geometry
0,5022,2480A0080,389,859,1248,"POLYGON ((20.32892 64.03356, 20.31428 64.03195..."
1,5023,2480A0090,221,495,716,"POLYGON ((20.64183 64.00944, 20.62268 64.00109..."
2,5030,2480C1350,407,308,715,"POLYGON ((20.31906 63.83298, 20.31904 63.83293..."
3,5031,2480C1390,231,388,619,"POLYGON ((20.28038 63.83529, 20.28018 63.83511..."
4,5032,2480C1250,308,577,885,"POLYGON ((20.25727 63.8278, 20.25704 63.82752,..."


## Läs in data

Vi använder GeoPandas för att läsa in geografiska data i minnet. De läggs i så kallade data frames. Speciella datastrukturer som består av tabeller av namngivna kolumner med data, som dessutom har en massa inbyggda funktioner för att manipulera data i kolumnerna.

In [ ]:

# ensure path is set earlier; normalize but do NOT rely on trailing slash
path = r"C:\Users\fabia\AH1023\Projekt\Projekt_0\Demo 2\data"
path = os.path.normpath(path)

# full filenames using os.path.join
shp1_path = os.path.join(path, "deso_b1_ume.shp")
shp2_path = os.path.join(path, "deso_a2_ume.shp")

print("Attempting to read:")
print("  ", shp1_path)
print("  ", shp2_path)

try:
    B1 = gpd.read_file(shp1_path)
    A2 = gpd.read_file(shp2_path)
except Exception as e:
    # show the real error for debugging
    raise RuntimeError(f"Failed to read shapefiles. Paths used: {shp1_path}, {shp2_path}. Error: {e}")

# Reproject to WGS84
B1.to_crs(4326, inplace=True)
A2.to_crs(4326, inplace=True)

def show_df(df, df_title):
    display(HTML("<h2>" + df_title + "</h2>"), df.head(), HTML('<br><br><br>'))

show_df(B1, "Befolkning")
show_df(A2, "Arbetande befolkning")
# ...existing code...

Attempting to read:
   C:\Users\fabia\AH1023\Projekt\Projekt 0\Demo 2\data\deso_b1_ume.shp
   C:\Users\fabia\AH1023\Projekt\Projekt 0\Demo 2\data\deso_a2_ume.shp


RuntimeError: Failed to read shapefiles. Paths used: C:\Users\fabia\AH1023\Projekt\Projekt 0\Demo 2\data\deso_b1_ume.shp, C:\Users\fabia\AH1023\Projekt\Projekt 0\Demo 2\data\deso_a2_ume.shp. Error: name 'gpd' is not defined

In [12]:
# Manipulera data

B1["olds"] = B1["alder65_w"] > B1["alder0_6"]

NameError: name 'B1' is not defined

## Visualisera zoner, noder och länkar

### Geopandas explore

Ibland händer det att explore() inte visar någon karta utan istället säger något om att notebooken inte är 'trusted'. Det är någon slags bug jag inte lyckats hitta en lösning på. Det brukar funka att spara, stänga och sedan öppna notebooken igen, så brukar kartan dyka upp.

In [11]:
B1.explore(column='olds')

NameError: name 'B1' is not defined

En intressant grej är att de områden där `B1["olds"]` är `True` visar sig vara relativt nybyggda områden. Redan här, med några rader kod kan vi börja dra slutsatser om rumsliga data.

### Kepler

Kepler är ett paket (ursprungligen utvecklat av Uber) som är bra för att visualisera stora geografiska dataset. Experimentera med att ändra hur data visualiseras genom att öppna menyn till vänster i kartan (en knapp märkt `<` för att expandera menyn). Det går till exempel att sätta att färgen ska ges av en av variablerna i datasetet. Kör nästa cell för att spara de inställningarna så att kepler-kartan ser likadan ut nästa gång notebooken körs. 

In [8]:

# use os.path.join for kepler config file (avoid string concat)
import os
config_path = os.path.join(path, "kepler_config.pkl")
# use os.path.join for kepler config file (avoid string concat)
config_path = os.path.join(path, "kepler_config.pkl")
config = pickle.load(open(config_path, 'rb'))

map = KeplerGl(height=920, config=config)    #Skapa en kepler karta.
map.add_data(data=B1.copy(), name='Zones')   #Lägg till en tabell som kartan kan använda.
map
# ...existing code...

# ...existing code...
# save kepler config using os.path.join
pickle.dump(map.config, open(os.path.join(path, "kepler_config.pkl"), "wb"))
# ...existing code...

NameError: name 'pickle' is not defined

## Spara ändringarna som gjorts i kepler i config-filen

In [ ]:
# Här kan man spara de ändringar som gjorts inne i kepler till config så de där där nästa gång kepler öppnas.

pickle.dump( map.config, open(path + "kepler_config.pkl", "wb" ) )